In [ ]:
import sys
sys.path.append('../Data_Feature')
sys.path.append('../Datasets')
sys.path.append('../Data_processing')
print(sys.path)
import numpy as np
import warnings
import os
import seaborn as sns
from scipy.stats import pearsonr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold, LeaveOneOut, cross_val_predict
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score, ConfusionMatrixDisplay, make_scorer, matthews_corrcoef, roc_curve, auc
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder 
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
import plotly.figure_factory as ff
import networkx as nx
from pyvis.network import Network
import zipfile 
from bioservices import KEGG
from K_func import translate_ko_terms
from data_processing import KOProcessor, GOProcessor

['c:\\Users\\eliah\\Documents\\Master\\Eliah-Masters\\Pipelines', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\\python312.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0', '', 'C:\\Users\\eliah\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages', 'C:\\Users\\eliah\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\win32', 'C:\\Users\\eliah\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\eliah

In [ ]:
# Import the custom data processing module
from data_processing import KOProcessor

# Paths to data files
terms_zip_path = '../Datasets/terms_KO.zip'
terms_csv_path = 'terms_KO.csv'
traits_reduced_zip_path = '../Datasets/reducedDataset.zip'
traits_reduced_csv_path = 'reducedDataset.csv'
traits_assembled_zip_path = '../Datasets/assembledDataset.zip'
traits_assembled_csv_path = 'assembledDataset.csv'

# Instantiate KOProcessor for loading and processing data
processor = KOProcessor(
    terms_zip_path=terms_zip_path,
    terms_csv_path=terms_csv_path,
    traits_reduced_zip_path=traits_reduced_zip_path,
    traits_reduced_csv_path=traits_reduced_csv_path,
    traits_assembled_zip_path=traits_assembled_zip_path,
    traits_assembled_csv_path=traits_assembled_csv_path
)

# Load KO terms and traits data
ko_terms = processor.load_terms()
reduced_traits_data = processor.load_reduced_traits_data()

# Preprocess KO terms and trophy traits
X_terms = processor.preprocess_terms(ko_terms)
y_traits = processor.preprocess_traits(reduced_traits_data, trait_column='trophy', use_assembled_if_missing=True)

# Align features and labels
X_aligned, y_aligned = processor.align_data(X_terms, y_traits)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_aligned, y_aligned, test_size=0.2, random_state=42)

# Initialize the model for multilabel classification
multioutput_clf = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))

# Train the model
multioutput_clf.fit(X_train, y_train)

# Make predictions
y_pred = multioutput_clf.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=y_aligned.columns))
